In [1]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Plots, Printf, IndexedGraphs, Statistics, Random
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using TensorTrains: summary_compact
using SparseArrays;

In [2]:
T = 25
N = 40
seed = 2

c = 2
gg = erdos_renyi(N, c/N; seed)
g = IndexedGraph(gg)

λ_unif = 0.5
ρ_unif = 0.2
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        λ[i] = rand()
    end
end
λ = sparse(λ)
ρ = rand(N)
γ = 0.5

# T = 3
# N = 2
# seed = 3

# A = [0 1; 1 0]
# g = IndexedGraph(A)

# λ_unif = 0.5
# ρ_unif = 0.6
# λ = sparse(λ_unif .* A)
# # λ = sparse([0 λ_unif; 0 0])
# ρ = fill(ρ_unif, N)
# γ = 0.5

sis = SIS_heterogeneous(λ, ρ, T; γ);
bp = mpbp(sis);

In [3]:
g.A

40×40 SparseMatrixCSC{Int64, Int64} with 70 stored entries:
⠀⠀⠀⠀⠀⠀⠁⠀⠀⠁⠀⠐⠀⠀⠂⠀⢀⡀⠀⠀
⠀⠀⠀⠀⠀⡀⢀⠀⠀⠀⠄⠀⠀⠀⠀⠀⢀⠀⠀⠀
⠀⠀⠀⠠⠀⠀⠀⠀⠠⠈⠒⡀⠀⢀⠄⠄⠈⠀⠀⠀
⠁⠀⠀⠐⠀⠀⠄⠁⠀⠃⢈⠀⠀⠀⠀⠄⠀⠀⠀⠀
⠄⠀⠀⠀⡀⠂⠤⠀⠀⠀⠀⠀⡀⠔⠀⠀⠀⠀⠄⠀
⢀⠀⠀⠁⠘⠠⠂⠐⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠈⠀
⠀⠀⠀⠀⠀⢀⠀⠀⢀⠌⠀⠀⠀⠀⠀⠀⠀⡀⠀⠀
⠈⠀⠀⠀⠀⠅⠀⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠠⢀⠠
⠀⠰⠀⠐⠂⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠂⠀⠀⠀⠀⡐⠀⠀⠀⠀

In [4]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [5]:
X

40×26 Matrix{Int64}:
 1  1  2  1  1  1  1  1  2  1  1  1  2  1  1  1  2  1  2  1  2  1  2  1  2  1
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  1  2  2  2  2  2  2  2  2  2  1  2  2  2  2  2  2  2  2  2  1  2  2
 2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 2  1  2  1  2  1  2  1  1  1  2  1  2  1  2  1  2  2  1  2  1  2  1  2  1  1
 1  2  1  1  1  1  1  2  1  1  1  2  1  1  1  1  1  1  2  1  1  1  1  1  1  1
 2  1  2  1  2  1  2  1  1  1  2  2  1  1  1  2  1  2  1  1  1  1  1  2  1  1
 2  1  2  1  2  1  2  1  1  2  1  2  1  1  2  1  1  2  1  1  1  2  1  1  1  1
 ⋮              ⋮              ⋮              ⋮              ⋮              ⋮
 2  2  2  2  1  1  2  2  2  2  2  2  1  2  1  2  2  2  2  2  1  2  2  2  2  2
 2  2  2  2  2  2  2  2  2  2  2  2  2  1  

In [6]:
reset_messages!(bp)
reset_beliefs!(bp)
svd_trunc = TruncBond(10)

iters, = iterate!(bp, maxiter=5; svd_trunc, tol=1e-12, damp=0.5);

Running MPBP: iter 2    Time: 0:01:32

In [7]:
obs_node = 6
for t in eachindex(bp.w[obs_node])
    bp.w[obs_node][t].ρ = ρ[obs_node]
end

der_mpbp = der_ρ(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

3.6195641682129613

In [8]:
ϵ = 1e-12

for i in vertices(g)
    bp.w[i][1].ρ = ρ[obs_node]
end

logzᵢ0 = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

bp.w[obs_node][1].ρ += ϵ

logzᵢϵ = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

der_num = logzᵢϵ/ϵ - logzᵢ0/ϵ 

3.6201171875